In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import pearsonr
import pandas as pd

import sys
sys.path.append('../../src/')
from coal_cov import *
from seg_sites_covar import CorrSegSites
from plot_utils import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['figure.facecolor'] = "w"
plt.rcParams['figure.autolayout'] = True
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker

import os
main_figdir = '../../plots/two_locus_stats/'
supp_figdir = '../../plots/supp_figs/two_locus_stats/'
os.makedirs(main_figdir, exist_ok=True)
os.makedirs(supp_figdir, exist_ok=True)

### Simulation under a model of exponential growth 

In [3]:
# NOTE : growth always starts ~ 1000 Generations in the past ... 
np.random.seed(42)

tas = [0, 100, 500]
rs = [1e-3, 1e-2, 5e-2]
rec_rate = 1e-3


total_sims = []
for t in tqdm(tas):
    for r in rs:
        cur_sim = TwoLocusSerialGrowth(ta=t, r=r, T=500, reps=50000, rec_rate=rec_rate)
        cur_sim._simulate()
        cur_sim._two_locus_branch_length()
        # Calculating the marginal variance and means
        mu_LA= np.mean(cur_sim.pair_branch_length[:,0])
        var_LA = np.var(cur_sim.pair_branch_length[:,0])
        cov_LALB = np.cov(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0,1]
        corr_LALB = pearsonr(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0]
        total_sims.append([t,r, mu_LA, var_LA, cov_LALB, corr_LALB])
            
total_sims = np.vstack(total_sims)

100%|██████████| 3/3 [03:48<00:00, 76.14s/it]


In [4]:
growth_df = pd.DataFrame(total_sims)
growth_df.columns = ['$t_a$', '$r$','$\mathbb{E}[L]$','$Var(L)$','$Cov(L_A,L_B)$','$Corr(L_A, L_B)$']
growth_df.to_csv('../../results/two_loci/growth_model.csv', index=False)
growth_df

,$t_a$,$r$,$\mathbb{E}[L]$,$Var(L)$,"$Cov(L_A,L_B)$","$Corr(L_A, L_B)$"
0,0.0,0.001,6090.662114,4.642052e+06,798884.492961,0.170881
1,0.0,0.010,1943.085588,6.517525e+04,20985.120789,0.319205
2,0.0,0.050,1253.275432,2.621982e+03,1003.661872,0.383109
3,100.0,0.001,6010.473723,4.634017e+06,689787.535148,0.149853
4,100.0,0.010,1846.649968,6.497690e+04,21237.949186,0.325849
5,100.0,0.050,1153.133224,2.613365e+03,1077.049356,0.412077
6,500.0,0.001,5685.082073,4.169165e+06,598531.835728,0.144119
7,500.0,0.010,1450.593252,5.948971e+04,24539.813168,0.413162
8,500.0,0.050,753.868166,2.522517e+03,1434.472770,0.566798


### Simulation under a bottleneck model

In [5]:
np.random.seed(42)

tas = [0, 100, 500]
rs = [1e-1, 1e-2, 1e-3]
rec_rate = 1e-3

total_sims_bot = []
for t in tqdm(tas):
    for r in rs:
        cur_sim = TwoLocusSerialBottleneck(Ne=1e4, ta=t, Tstart=50, Tend=500, Nbot=r*1e4, reps=50000, rec_rate=rec_rate)
        cur_sim._simulate()
        cur_sim._two_locus_branch_length()
        # Calculating the marginal variance and means
        mu_LA= np.mean(cur_sim.pair_branch_length[:,0])
        var_LA = np.var(cur_sim.pair_branch_length[:,0])
        cov_LALB = np.cov(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0,1]
        corrLALB = pearsonr(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0]
        total_sims_bot.append([t, r, mu_LA, var_LA, cov_LALB, corrLALB])
        
        
total_sims_bot = np.vstack(total_sims_bot)

100%|██████████| 3/3 [03:35<00:00, 71.71s/it]


In [6]:
bottleneck_df = pd.DataFrame(total_sims_bot)
bottleneck_df.columns = ['$t_a$', '$\phi$','$\mathbb{E}[L]$','$Var(L)$','$Cov(L_A,L_B)$','$Corr(L_A,L_B)$']
bottleneck_df.to_csv('../../results/two_loci/bottleneck.csv', index=False)
bottleneck_df

,$t_a$,$\phi$,$\mathbb{E}[L]$,$Var(L)$,"$Cov(L_A,L_B)$","$Corr(L_A,L_B)$"
0,0.0,0.100,33273.741545,1.568670e+09,1.847574e+08,0.118042
1,0.0,0.010,4701.671637,3.392016e+08,1.092700e+08,0.327205
2,0.0,0.001,139.699778,1.643994e+03,1.501318e+03,0.911499
3,100.0,0.100,33502.508156,1.565242e+09,1.667204e+08,0.106000
4,100.0,0.010,5982.054339,4.163824e+08,1.400499e+08,0.344304
5,100.0,0.001,140.175515,1.612585e+03,1.552592e+03,0.962495
6,500.0,0.100,40311.226627,1.568729e+09,3.864358e+07,0.024572
7,500.0,0.010,40443.725126,1.594782e+09,3.452399e+07,0.021530
8,500.0,0.001,40498.188111,1.615431e+09,4.759060e+07,0.029511
